In [1]:
import sys
!{sys.executable} -m pip install --upgrade keras

    100% |████████████████████████████████| 337kB 2.8MB/s ta 0:00:01
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python3.6/dist-packages (from keras)
    100% |████████████████████████████████| 12.2MB 122kB/s eta 0:00:01
  Using cached six-1.11.0-py2.py3-none-any.whl
  Found existing installation: numpy 1.14.0
    Uninstalling numpy-1.14.0:
      Successfully uninstalled numpy-1.14.0
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: PyYAML 3.11
    Uninstalling PyYAML-3.11:
      Successfully uninstalled PyYAML-3.11
  Found existing installation: Keras 2.0.6
    Uninstalling Keras-2.0.6:
      Successfully uninstalled Keras-2.0.6


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

EMBEDDING_FILE='/public/models/glove/glove.840B.300d.txt'
TRAIN_DATA_FILE='/public/toxic_comments/train.csv'
TEST_DATA_FILE='/public/toxic_comments/test.csv'

MAX_SEQUENCE_LENGTH = 150
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300
num_dense = 256
rate_drop_lstm = 0.2
rate_drop_dense = 0.2

act = 'relu'

In [4]:
def cleanData(text, stemming = False, lemmatize=False):    
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])
    return text

In [5]:
print('Indexing word vectors')

count = 0
embeddings_index = {}
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs.reshape(-1)
    coef = embeddings_index[word]
f.close()

print('Found %d word vectors of glove.' % len(embeddings_index))
emb_mean,emb_std = coef.mean(), coef.std()
print(emb_mean,emb_std)

print('Total %s word vectors.' % len(embeddings_index))

train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

Indexing word vectors
Found 2195895 word vectors of glove.
-0.01444638 0.47249147
Total 2195895 word vectors.


In [6]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers import concatenate, CuDNNGRU
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys

In [7]:
import re

print('Processing text dataset')

train_df['comment_text'] = train_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                            lemmatize=False))
test_df['comment_text'] = test_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                          lemmatize=False))

#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)
#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    return(text)


list_sentences_train = train_df["comment_text"].fillna("NA").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("NA").values


comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))
    
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Processing text dataset
Found 292462 unique tokens
Shape of data tensor: (159571, 150)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 150)


In [8]:
data_post = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post', truncating='post')
print('Shape of data tensor:', data_post.shape)
print('Shape of label tensor:', y.shape)

test_data_post = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of test_data tensor:', test_data_post.shape)

Shape of data tensor: (159571, 150)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 150)


In [9]:
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 21603


In [10]:
max_features=100000
maxlen=150
embed_size=300

In [11]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [12]:
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale


# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [13]:
import sys
from os.path import dirname
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K


class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [14]:
from keras.optimizers import Adam

In [15]:
def get_model():
    input1_pre = Input(shape=(maxlen,))
    embed_layer1_pre = Embedding(max_features,
                            embed_size,
                            input_length=maxlen,
                            weights=[embedding_matrix],
                            trainable=False)(input1_pre)
    embed_layer1_pre = SpatialDropout1D(0.4)(embed_layer1_pre)
    
    x_pre = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer1_pre)
    capsule_pre = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x_pre)
    # output_capsule = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule)
    capsule_pre = GlobalMaxPooling1D()(capsule_pre)
    capsule_pre = Dropout(0.25)(capsule_pre)
    
    input1_post = Input(shape=(maxlen,))
    embed_layer1_post = Embedding(max_features,
                            embed_size,
                            input_length=maxlen,
                            weights=[embedding_matrix],
                            trainable=False)(input1_post)
    embed_layer1_post = SpatialDropout1D(0.4)(embed_layer1_post)
    
    x_post = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer1_post)
    capsule_post = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x_post)
    #  output_capsule = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule)
    capsule_post = GlobalMaxPooling1D()(capsule_post)
    capsule_post = Dropout(0.25)(capsule_post)
    
    concat = concatenate([capsule_pre,capsule_post])
    output = Dense(6, activation='sigmoid')(concat)
    
    model = Model(inputs=[input1_pre,input1_post], outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=1e-3,decay=0),
        metrics=['accuracy'])
#     model.summary()
    return model

In [16]:
from sklearn.metrics import log_loss
import numpy as np

test_predicts_list = []

def train_folds(data,data_post, y,fold_count=10):
    print("Starting to train models...")
    fold_size = len(data) // fold_count
    models = []
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(data)

        print("Fold {0}".format(fold_id))
        
        train_x = np.concatenate([data[:fold_start], data[fold_end:]])
        train_xp = np.concatenate([data_post[:fold_start], data_post[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = data[fold_start:fold_end]
        val_xp = data_post[fold_start:fold_end]
        val_y = y[fold_start:fold_end]
        
        file_path="capsule_fold{0}.h5".format(fold_id)
        model = get_model()
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
        RocAuc = RocAucEvaluation(validation_data=([val_x, val_xp], val_y), interval=1)
        callbacks_list = [checkpoint, early,RocAuc] 

        hist = model.fit([train_x, train_xp], train_y, epochs=15, batch_size=128, shuffle=True, 
                         validation_data=([val_x, val_xp], val_y), callbacks = callbacks_list, verbose=1)
        model.load_weights(file_path)
        best_score = min(hist.history['val_loss'])
        
        print("Fold {0} loss {1}".format(fold_id, best_score))
        print("Predicting validation...")
        val_predicts_path = "capsule_val_predicts{0}.npy".format(fold_id)
        val_predicts = model.predict([val_x, val_xp], batch_size=1024, verbose=1)
        np.save(val_predicts_path, val_predicts)
        
        print("Predicting results...")
        test_predicts_path = "capsule_test_predicts{0}.npy".format(fold_id)
        test_predicts = model.predict([test_data, test_data_post], batch_size=1024, verbose=1)
        test_predicts_list.append(test_predicts)
        np.save(test_predicts_path, test_predicts)

In [17]:
train_folds(data, data_post, y)

Starting to train models...
Fold 0
Train on 143614 samples, validate on 15957 samples
Epoch 1/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.1001 - acc: 0.9751 - val_loss: 0.0490 - val_acc: 0.9825

Epoch 00001: val_loss improved from inf to 0.04905, saving model to capsule_fold0.h5

 ROC-AUC - epoch: 1 - score: 0.977384
Epoch 2/15
143614/143614 [==============================] - 209s 1ms/step - loss: 0.0511 - acc: 0.9819 - val_loss: 0.0449 - val_acc: 0.9829

Epoch 00002: val_loss improved from 0.04905 to 0.04492, saving model to capsule_fold0.h5

 ROC-AUC - epoch: 2 - score: 0.983373
Epoch 3/15
143614/143614 [==============================] - 209s 1ms/step - loss: 0.0462 - acc: 0.9829 - val_loss: 0.0430 - val_acc: 0.9834

Epoch 00003: val_loss improved from 0.04492 to 0.04298, saving model to capsule_fold0.h5

 ROC-AUC - epoch: 3 - score: 0.982130
Epoch 4/15
143614/143614 [==============================] - 209s 1ms/step - loss: 0.0438 - acc: 0.9835 - val_lo

143614/143614 [==============================] - 209s 1ms/step - loss: 0.0399 - acc: 0.9846 - val_loss: 0.0394 - val_acc: 0.9843

Epoch 00007: val_loss improved from 0.03948 to 0.03944, saving model to capsule_fold2.h5

 ROC-AUC - epoch: 7 - score: 0.990547
Epoch 8/15
143614/143614 [==============================] - 208s 1ms/step - loss: 0.0381 - acc: 0.9852 - val_loss: 0.0389 - val_acc: 0.9841

Epoch 00008: val_loss improved from 0.03944 to 0.03890, saving model to capsule_fold2.h5

 ROC-AUC - epoch: 8 - score: 0.991079
Epoch 9/15
143614/143614 [==============================] - 209s 1ms/step - loss: 0.0370 - acc: 0.9855 - val_loss: 0.0395 - val_acc: 0.9839

Epoch 00009: val_loss did not improve

 ROC-AUC - epoch: 9 - score: 0.991103
Epoch 10/15
143614/143614 [==============================] - 209s 1ms/step - loss: 0.0360 - acc: 0.9858 - val_loss: 0.0392 - val_acc: 0.9840

Epoch 00010: val_loss did not improve

 ROC-AUC - epoch: 10 - score: 0.990921
Epoch 11/15
143614/143614 [========


 ROC-AUC - epoch: 14 - score: 0.990172
Fold 4 loss 0.03870266953554079
Predicting validation...
15957/15957 [==============================] - 4s 263us/step
Predicting results...
153164/153164 [==============================] - 39s 252us/step
Fold 5
Train on 143614 samples, validate on 15957 samples
Epoch 1/15
143614/143614 [==============================] - 213s 1ms/step - loss: 0.0884 - acc: 0.9772 - val_loss: 0.0472 - val_acc: 0.9835

Epoch 00001: val_loss improved from inf to 0.04715, saving model to capsule_fold5.h5

 ROC-AUC - epoch: 1 - score: 0.975727
Epoch 2/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0498 - acc: 0.9821 - val_loss: 0.0426 - val_acc: 0.9837

Epoch 00002: val_loss improved from 0.04715 to 0.04264, saving model to capsule_fold5.h5

 ROC-AUC - epoch: 2 - score: 0.983071
Epoch 3/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0475 - acc: 0.9824 - val_loss: 0.0425 - val_acc: 0.9835

Epoch 00003: val_loss i

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



143614/143614 [==============================] - 211s 1ms/step - loss: 0.0370 - acc: 0.9855 - val_loss: 0.0386 - val_acc: 0.9847

Epoch 00009: val_loss did not improve

 ROC-AUC - epoch: 9 - score: 0.988935
Epoch 10/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0361 - acc: 0.9858 - val_loss: 0.0383 - val_acc: 0.9848

Epoch 00010: val_loss improved from 0.03829 to 0.03826, saving model to capsule_fold5.h5

 ROC-AUC - epoch: 10 - score: 0.988859
Epoch 11/15
126336/143614 [=========================>....] - ETA: 24s - loss: 0.0349 - acc: 0.9863

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



143614/143614 [==============================] - 210s 1ms/step - loss: 0.0467 - acc: 0.9827 - val_loss: 0.0421 - val_acc: 0.9837

Epoch 00003: val_loss improved from 0.04421 to 0.04215, saving model to capsule_fold6.h5

 ROC-AUC - epoch: 3 - score: 0.985340
Epoch 4/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0438 - acc: 0.9832 - val_loss: 0.0407 - val_acc: 0.9841

Epoch 00004: val_loss improved from 0.04215 to 0.04072, saving model to capsule_fold6.h5

 ROC-AUC - epoch: 4 - score: 0.985515
Epoch 5/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0423 - acc: 0.9838 - val_loss: 0.0397 - val_acc: 0.9844

Epoch 00005: val_loss improved from 0.04072 to 0.03971, saving model to capsule_fold6.h5

 ROC-AUC - epoch: 5 - score: 0.987021
Epoch 6/15
 47232/143614 [========>.....................] - ETA: 2:15 - loss: 0.0411 - acc: 0.9840

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



143614/143614 [==============================] - 211s 1ms/step - loss: 0.0509 - acc: 0.9819 - val_loss: 0.0453 - val_acc: 0.9827

Epoch 00002: val_loss improved from 0.04908 to 0.04534, saving model to capsule_fold7.h5

 ROC-AUC - epoch: 2 - score: 0.982731
Epoch 3/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0466 - acc: 0.9828 - val_loss: 0.0426 - val_acc: 0.9837

Epoch 00003: val_loss improved from 0.04534 to 0.04258, saving model to capsule_fold7.h5

 ROC-AUC - epoch: 3 - score: 0.984120
Epoch 4/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0435 - acc: 0.9837 - val_loss: 0.0416 - val_acc: 0.9838

Epoch 00004: val_loss improved from 0.04258 to 0.04156, saving model to capsule_fold7.h5

 ROC-AUC - epoch: 4 - score: 0.985784
Epoch 5/15
143614/143614 [==============================] - 210s 1ms/step - loss: 0.0414 - acc: 0.9842 - val_loss: 0.0420 - val_acc: 0.9836

Epoch 00005: val_loss did not improve

 ROC-AUC - epoch: 5 - sc


 ROC-AUC - epoch: 6 - score: 0.987210
Epoch 7/15
143614/143614 [==============================] - 211s 1ms/step - loss: 0.0401 - acc: 0.9845 - val_loss: 0.0419 - val_acc: 0.9834

Epoch 00007: val_loss did not improve

 ROC-AUC - epoch: 7 - score: 0.988784
Epoch 8/15
143614/143614 [==============================] - 212s 1ms/step - loss: 0.0388 - acc: 0.9849 - val_loss: 0.0402 - val_acc: 0.9841

Epoch 00008: val_loss improved from 0.04188 to 0.04024, saving model to capsule_fold9.h5

 ROC-AUC - epoch: 8 - score: 0.989455
Epoch 9/15
143614/143614 [==============================] - 212s 1ms/step - loss: 0.0374 - acc: 0.9853 - val_loss: 0.0402 - val_acc: 0.9842

Epoch 00009: val_loss improved from 0.04024 to 0.04020, saving model to capsule_fold9.h5

 ROC-AUC - epoch: 9 - score: 0.988745
Epoch 10/15
143614/143614 [==============================] - 212s 1ms/step - loss: 0.0365 - acc: 0.9857 - val_loss: 0.0405 - val_acc: 0.9839

Epoch 00010: val_loss did not improve

 ROC-AUC - epoch: 10 - s

In [18]:
CLASSES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print(len(test_predicts_list))
test_predicts_am = np.zeros(test_predicts_list[0].shape)

for fold_predict in test_predicts_list:
    test_predicts_am += fold_predict

test_predicts_am = (test_predicts_am / len(test_predicts_list))

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts_am = pd.DataFrame(data=test_predicts_am, columns=CLASSES)
test_predicts_am["id"] = test_ids
test_predicts_am = test_predicts_am[["id"] + CLASSES]
test_predicts_am.to_csv("10fold_capsule_am.csv", index=False)

10


In [ ]:
test_predicts = np.ones(test_predicts_list[0].shape)

for fold_predict in test_predicts_list:
    test_predicts *= fold_predict

test_predicts **= (1. / len(test_predicts_list))

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
test_predicts.to_csv("10fold_capsule_gm.csv", index=False)